In [2]:
import pandas as pd
import os
import copy
import numpy as np

### 查看结果的方法

   1. 解注释下方的cross_val变量
   2. 然后依次运行下面代码，即可看到最后结果
   3. shift+回车键 快捷执行代码

In [66]:
model = "RF"   # RF LR MLP KNN可选

cross_val = "intra_subject"
#cross_val = "LOSO"

#task = "binary"
task = "tenary"

base_dir = f'./all_result/{model}'
cur_dir = os.path.join(os.path.join(base_dir, cross_val), task)
rs_dir = os.listdir(cur_dir)
rs_dir_copy = copy.deepcopy(rs_dir)
for rid, rd in enumerate(rs_dir_copy):
    if '.' in rd:
        rs_dir.pop(rid)
print(rs_dir)

['2', '3', '1', '4']


In [67]:
PW0_mean = np.zeros((len(rs_dir), 4), dtype=float)
PW0_std = np.zeros((len(rs_dir), 4), dtype=float)
for rsd in rs_dir:
    PW0_mean[int(rsd)-1, 0] = int(rsd)
    PW0_std[int(rsd)-1, 0] = int(rsd)
    
    cur_rs_dir = os.path.join(cur_dir, rsd)
    all_rs = os.listdir(cur_rs_dir)
    PW_0_mean = []
    PW_0_std = []
    
    for cur_file in all_rs:
        if not str(cur_file).endswith(".csv"):
            continue
        cur_file_dir = os.path.join(cur_rs_dir, cur_file)
        #pw_i = int(str(cur_file).split(".")[0][-1])
        if cross_val != "LOSO":
            # read intra-sub result out
            rs = np.loadtxt(cur_file_dir, dtype=float, delimiter=",", skiprows=0, usecols=[0, 2, 4])
            # rs = pd.read_csv(cur_file_dir, sep=",", names=["AUC_ROC", "Kappa", "Acc", "macro_P", "weight_P", "f1"])
        else:
            # read loso result out
            rs = np.loadtxt(cur_file_dir, dtype=float, delimiter=",", skiprows=1, usecols=[0, 2, 4])
            # rs = pd.read_csv(cur_file_dir, sep=",", names=["AUC_ROC", "Kappa", "Acc", "macro_P", "weight_P", "f1"], header=0)
            
        mean = np.mean(rs, axis=0, keepdims=True)
        std = np.std(rs, axis=0, keepdims=True)
        
        PW_0_mean.append(mean)
        PW_0_std.append(std)
    
    print("\n")
    print(f"当前窗口为：{rsd}s")    
    print(pd.DataFrame(np.concatenate(PW_0_mean, axis=0), columns=['AUC_ROC', 'Acc', 'F1-score'], dtype=float))
        
    PW0_mean[int(rsd)-1, 1:] = np.mean(np.concatenate(PW_0_mean, axis=0), axis=0, keepdims=True)[0]
    PW0_std[int(rsd)-1, 1:] = np.std(np.concatenate(PW_0_std, axis=0), axis=0, keepdims=True)[0] if cross_val != "LOSO" else np.concatenate(PW_0_std, axis=0)



当前窗口为：2s
     AUC_ROC       Acc  F1-score
0   1.000000  1.000000  1.000000
1   1.000000  1.000000  1.000000
2   1.000000  0.996666  0.996774
3   1.000000  1.000000  1.000000
4   1.000000  1.000000  1.000000
5   1.000000  1.000000  1.000000
6   1.000000  0.996552  0.996666
7   1.000000  1.000000  1.000000
8   1.000000  1.000000  1.000000
9   1.000000  0.993334  0.993750
10  1.000000  1.000000  1.000000
11  1.000000  1.000000  1.000000
12  1.000000  1.000000  1.000000
13  1.000000  1.000000  1.000000
14  1.000000  1.000000  1.000000
15  1.000000  1.000000  1.000000
16  1.000000  1.000000  1.000000
17  1.000000  1.000000  1.000000
18  1.000000  1.000000  1.000000
19  1.000000  1.000000  1.000000
20  1.000000  1.000000  1.000000
21  1.000000  1.000000  1.000000
22  1.000000  0.996666  0.996774
23  1.000000  1.000000  1.000000
24  1.000000  1.000000  1.000000
25  1.000000  1.000000  1.000000
26  1.000000  1.000000  1.000000
27  1.000000  1.000000  1.000000
28  1.000000  1.000000  1.000000

In [68]:
rs = [PW0_mean, PW0_std]
rs_name = ["mean","std"]
for name, content in zip(rs_name, rs):
    print(f"--------------------------{name}-----------------------------")
    c = pd.DataFrame(content, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)
    print(c)
    print("\n")

--------------------------mean-----------------------------
    SW   AUC_ROC       Acc  F1-score
0  1.0  0.989072  0.988516  0.988558
1  2.0  0.999812  0.997844  0.998031
2  3.0  0.999567  0.996211  0.996704
3  4.0  0.999832  0.996254  0.996748


--------------------------std-----------------------------
    SW   AUC_ROC       Acc  F1-score
0  1.0  0.002602  0.005408  0.005421
1  2.0  0.001761  0.007592  0.006620
2  3.0  0.002381  0.009149  0.007302
3  4.0  0.001917  0.010244  0.008403




In [69]:
pd.DataFrame(PW0_mean, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)

,SW,AUC_ROC,Acc,F1-score
0,1.0,0.989072,0.988516,0.988558
1,2.0,0.999812,0.997844,0.998031
2,3.0,0.999567,0.996211,0.996704
3,4.0,0.999832,0.996254,0.996748


In [70]:
pd.DataFrame(PW0_std, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)

,SW,AUC_ROC,Acc,F1-score
0,1.0,0.002602,0.005408,0.005421
1,2.0,0.001761,0.007592,0.006620
2,3.0,0.002381,0.009149,0.007302
3,4.0,0.001917,0.010244,0.008403
